In [6]:
using Gumbo
using Cascadia
using DataFrames
using JuMP
using HiGHS
using CSV


┌ Warning: Module Parsers with build ID 723101737944901 is missing from the cache.
│ This may mean Parsers [69de0a69-1ddd-5017-9359-2bf0b02dc9f0] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107


┌ Warning: Module Parsers with build ID 723101737944901 is missing from the cache.
│ This may mean Parsers [69de0a69-1ddd-5017-9359-2bf0b02dc9f0] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107


┌ Warning: Module DataAPI with build ID 788775425341301 is missing from the cache.
│ This may mean DataAPI [9a962f9c-6df0-11e9-0e5d-c546b8b5ee8a] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107


┌ Warning: Module Compat with build ID 788775090396201 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107


┌ Warning: Module Preferences with build ID 723103955724101 is missing from the cache.
│ This may mean Preferences [21216c6a-2e73-6563-6e65-726566657250] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107


In [ ]:
# import timeseries data 


In [ ]:
model = Model(HiGHS.Optimizer)